In [ ]:
# Install LlamaIndex core + HuggingFace embeddings

!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface
!pip install transformers accelerate bitsandbytes


#!pip install llama-index sentence-transformers

In [ ]:

from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoModelForCausalLM, AutoTokenizer


I THINK I MIGHT NEED TO DELETE IT

In [ ]:
import os
from google.colab import userdata
from transformers import BitsAndBytesConfig
print(userdata.get('HF_TOKEN'))

# Use a model
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Define quantization config
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16, # Assuming you have a GPU
)

hf_llm = HuggingFaceLLM(model_name= model_name,
                        tokenizer_name= model_name,
                        device_map="auto",
                        #model_kwargs={"load_in_4bit": True} # Deprecated
                        model_kwargs={"quantization_config": nf4_config}
                        )

# Example small dataset
documents = [Document(text="This is a small medical dataset.")]

# Use a local HF embedding model (no OpenAI key needed)
#embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

hf_embed = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.embed_model = hf_embed
#Settings.embed_model = embed_model
Settings.llm = hf_llm




# Example small dataset
documents = [Document(text="This is a small medical dataset.")]


# Build the vector index
index = VectorStoreIndex.from_documents(documents)

# Query the index
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query("What is this dataset about?")
print(response)

In [ ]:
!pip install llama-index-vector-stores-faiss


In [ ]:
from llama_index.vector_stores.faiss import FaissVectorStore


In [ ]:
!pip install faiss-gpu tqdm llama-index-vector-stores-faiss

import os, glob, torch, faiss
from math import ceil
from tqdm import tqdm
from google.colab import drive
from llama_index.core import Document, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SimpleNodeParser

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ Paths
folder_path = "/content/drive/My Drive/datasets/med-dialog/large-english-dialog-corpus/*.txt"
faiss_path = "/content/drive/My Drive/datasets/med-dialog/faiss_index"

# ✅ Load documents
documents = []
for file in glob.glob(folder_path):
    with open(file, "r", encoding="utf-8") as f:
        documents.append(Document(text=f.read().strip()))

print(f"📂 Total documents: {len(documents)}")

# ✅ Detect GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# ✅ Hugging Face embeddings (GPU if available)
hf_embed = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    device=device
)
Settings.embed_model = hf_embed
Settings.llm = None  # Disable LLM for indexing

# ✅ Chunk large documents
parser = SimpleNodeParser.from_defaults(chunk_size=512, chunk_overlap=50)
all_nodes = parser.get_nodes_from_documents(documents)
print(f"✅ After chunking: {len(all_nodes)} chunks generated from {len(documents)} docs")

# ✅ FAISS initialization
embedding_dim = hf_embed._model.get_sentence_embedding_dimension()
faiss_index = faiss.IndexFlatL2(embedding_dim)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# ✅ Batch processing with progress bar
batch_size = 64  # larger batch size since we're chunking
num_nodes = len(all_nodes)
num_batches = ceil(num_nodes / batch_size)
print(f"🔄 Starting indexing in {num_batches} batches (batch size = {batch_size})...")

for i in tqdm(range(num_batches), desc="📌 Indexing batches"):
    start = i * batch_size
    end = min(start + batch_size, num_nodes)
    batch_nodes = all_nodes[start:end]

    VectorStoreIndex.from_documents(batch_nodes, embed_model=hf_embed, storage_context=storage_context)

    # ✅ Save progress
    storage_context.persist(persist_dir=faiss_path)

print("🎉 All chunks indexed and saved successfully!")


In [ ]:
import os, glob, torch
from math import ceil
from google.colab import drive, userdata
from llama_index.core import Document, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from transformers import BitsAndBytesConfig
from llama_index.llms.huggingface import HuggingFaceLLM

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ Paths
folder_path = "/content/drive/My Drive/datasets/med-dialog/large-english-dialog-corpus/*.txt"
faiss_path = "/content/drive/My Drive/datasets/med-dialog/faiss_index"

# ✅ Load all documents
documents = []
for file in glob.glob(folder_path):
    with open(file, "r", encoding="utf-8") as f:
        documents.append(Document(text=f.read().strip()))

print(f"📂 Total documents: {len(documents)}")

# ✅ Hugging Face embeddings
hf_embed = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2",

                                )
Settings.embed_model = hf_embed
Settings.llm = None  # Disable LLM during indexing for speed



In [ ]:
!pip install faiss-cpu


In [ ]:
import faiss

from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import StorageContext

# Get embedding dimension
embedding_dim = hf_embed._model.get_sentence_embedding_dimension()

# ✅ Create FAISS L2 index
faiss_index = faiss.IndexFlatL2(embedding_dim)

# Create a new FAISS store
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
import torch
print(torch.cuda.is_available())


In [ ]:
import time
start = time.time()
_ = hf_embed.get_text_embedding(documents[0].text)
print("⏱️ Embedding time for 1 doc:", time.time()-start, "seconds")


In [ ]:
# ✅ Batch processing
batch_size = 32
num_docs = len(documents)
num_batches = ceil(num_docs / batch_size)

print(f"🔄 Starting batch indexing: {num_batches} batches (batch size = {batch_size})")

for i in tqdm(range(num_batches), desc="📌 Indexing batches"):
    start = i * batch_size
    end = min(start + batch_size, num_docs)
    batch_docs = documents[start:end]

    print(f"📌 Batch {i+1}/{num_batches} → Documents {start+1}-{end}")
    VectorStoreIndex.from_documents(batch_docs, embed_model=hf_embed, storage_context=storage_context)

    # ✅ Save after each batch
    storage_context.persist(persist_dir=faiss_path)
    print(f"✅ Batch {i+1} saved to {faiss_path}")

print("🎉 All batches indexed and saved successfully!")








In [ ]:
# ✅ Initialize FAISS vector store
vector_store = FaissVectorStore(dim=hf_embed._model.get_sentence_embedding_dimension())
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# ✅ Batching setup
batch_size = 32
num_docs = len(documents)
num_batches = ceil(num_docs / batch_size)

print(f"🔄 Processing {num_batches} batches (batch size = {batch_size})...")

# ✅ Process in batches and save after each
for i in range(num_batches):
    start = i * batch_size
    end = min(start + batch_size, num_docs)
    batch_docs = documents[start:end]

    print(f"📌 Batch {i+1}/{num_batches} ({start+1}-{end}) indexing...")
    VectorStoreIndex.from_documents(batch_docs, embed_model=hf_embed, storage_context=storage_context)

    # ✅ Persist after each batch
    storage_context.persist(persist_dir=faiss_path)
    print(f"✅ Batch {i+1}/{num_batches} saved to {faiss_path}")

print("🎉 All batches indexed and saved!")


In [ ]:
# ✅ TASK 1: Mount Google Drive
print("🔄 TASK 1: Mounting Google Drive...")
drive.mount('/content/drive')
print("✅ TASK 1 Completed: Drive mounted.")

# ✅ TASK 2: Load text files
print("🔄 TASK 2: Loading dataset files...")
folder_path = "/content/drive/My Drive/datasets/med-dialog/large-english-dialog-corpus/*.txt"


In [ ]:
import glob, os
from llama_index.core import Document
from google.colab import drive


documents = []
for file in glob.glob(folder_path):
    with open(file, "r", encoding="utf-8") as f:
        text = f.read().strip()
        documents.append(Document(text=text))

print(f"✅ TASK 2 Completed: Loaded {len(documents)} documents.")

# Collect all text files
files = glob.glob(folder_path)

print(f"📂 Found {len(files)} documents in the folder.")

# Optionally, list first few file names
for f in files[:5]:
    print("➡️", f)

# ✅ TASK 3: Preview
if documents:
    print("🔄 TASK 3: Showing document previews...")
    for i, doc in enumerate(documents[:6]):
        print(f"\n--- Document {i+1} preview ---\n", doc.text[:300])
    print("✅ TASK 3 Completed: Preview displayed.")
else:
    print("⚠️ No documents found. Check folder path or file extensions.")


In [ ]:
import os, glob, torch
from math import ceil
from google.colab import drive, userdata
from llama_index.core import Document, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from transformers import BitsAndBytesConfig
from llama_index.llms.huggingface import HuggingFaceLLM

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ Paths
folder_path = "/content/drive/My Drive/datasets/med-dialog/large-english-dialog-corpus/*.txt"
faiss_path = "/content/drive/My Drive/datasets/med-dialog/faiss_index"

# ✅ Load all documents
documents = []
for file in glob.glob(folder_path):
    with open(file, "r", encoding="utf-8") as f:
        documents.append(Document(text=f.read().strip()))

print(f"📂 Total documents: {len(documents)}")

# ✅ Hugging Face embeddings
hf_embed = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.embed_model = hf_embed
Settings.llm = None  # Disable LLM during indexing for speed

# ✅ Initialize FAISS vector store
vector_store = FaissVectorStore(dim=hf_embed._model.get_sentence_embedding_dimension())
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# ✅ Batching setup
batch_size = 32
num_docs = len(documents)
num_batches = ceil(num_docs / batch_size)

print(f"🔄 Processing {num_batches} batches (batch size = {batch_size})...")

# ✅ Process in batches and save after each
for i in range(num_batches):
    start = i * batch_size
    end = min(start + batch_size, num_docs)
    batch_docs = documents[start:end]

    print(f"📌 Batch {i+1}/{num_batches} ({start+1}-{end}) indexing...")
    VectorStoreIndex.from_documents(batch_docs, embed_model=hf_embed, storage_context=storage_context)

    # ✅ Persist after each batch
    storage_context.persist(persist_dir=faiss_path)
    print(f"✅ Batch {i+1}/{num_batches} saved to {faiss_path}")

print("🎉 All batches indexed and saved!")


In [ ]:
import os, glob, torch
from google.colab import drive, userdata
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from transformers import BitsAndBytesConfig

# ✅ TASK 1: Mount Google Drive
print("🔄 TASK 1: Mounting Google Drive...")
drive.mount('/content/drive')
print("✅ TASK 1 Completed: Drive mounted.")

# ✅ TASK 2: Load dataset from Drive
folder_path = "/content/drive/My Drive/datasets/med-dialog/large-english-dialog-corpus/*.txt"
documents = []
for file in glob.glob(folder_path):
    with open(file, "r", encoding="utf-8") as f:
        text = f.read().strip()
        documents.append(Document(text=text))

print(f"✅ TASK 2 Completed: Loaded {len(documents)} documents.")

# ✅ Count and preview
files = glob.glob(folder_path)
print(f"📂 Found {len(files)} documents in the folder.")
for f in files[:5]:
    print("➡️", f)

if documents:
    print("🔄 TASK 3: Showing document previews...")
    for i, doc in enumerate(documents[:3]):
        print(f"\n--- Document {i+1} preview ---\n", doc.text[:300])
    print("✅ TASK 3 Completed: Preview displayed.")
else:
    print("⚠️ No documents found. Check folder path or file extensions.")

# ✅ TASK 4: Setup Hugging Face Token
hf_token = userdata.get('HF_TOKEN')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token
print("✅ TASK 4 Completed: Hugging Face token set.")

# ✅ TASK 5: Configure Mistral 7B with 4-bit quantization
print("🔄 TASK 5: Loading Mistral 7B model...")
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)
hf_llm = HuggingFaceLLM(
    model_name=model_name,
    tokenizer_name=model_name,
    device_map="auto",
    model_kwargs={"quantization_config": nf4_config}
)
print("✅ TASK 5 Completed: Mistral 7B loaded.")

# ✅ TASK 6: Setup Hugging Face embeddings
hf_embed = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.embed_model = hf_embed
Settings.llm = hf_llm
print("✅ TASK 6 Completed: Embeddings configured.")

# ✅ TASK 7: Build the vector index
print("🔄 TASK 7: Building vector index...")
index = VectorStoreIndex.from_documents(documents,embed_model=hf_embed, batch_size=32)
print("✅ TASK 7 Completed: Vector index built.")

# ✅ TASK 8: Test a sample query
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query("What topics are covered in this medical dataset?")
print("🔍 Query Answer:\n", response)


In [ ]:
%who


In [ ]:
index.storage_context.persist(persist_dir="/content/drive/My Drive/datasets/med-dialog/faiss_index")
print("✅ Index saved to Drive")
